In [2]:
import pandas as pd

In [ ]:
csvfile = "/Users/alifabdullah/Collaboration/Kaggle-ML-Algorithm-Musings/datasets/diabetes.csv"
csv_in_panda_form = pd.read_csv(csvfile)
target_column = "Outcome"
feature_columns = [feature_header for feature_header in csv_in_panda_form.columns.drop(target_column)]

#print(csv_in_panda_form)
#print(feature_columns)

for row in csv_in_panda_form.iterrows():
  print(row[1]["Outcome"])
  
  


In [26]:
def dataset_consistency_metrics(dataframe, list_of_labels, target_column):
  label_count_mapping_dict = dict()
  for key in list_of_labels:
    label_count_mapping_dict[key] = 0
  
  for row in dataframe.iterrows():
    label_count_mapping_dict[row[1][target_column]] += 1 
  return label_count_mapping_dict

dataset_consistency_metrics(csv_in_panda_form, [0,1], "Outcome")

{0: 500, 1: 268}